# read data

In [1]:
import pandas as pd

PATH = "/home/kai/data/kaggle/talkingdata/data/"
test_nrows = 5
nrows = None
dtypes = {
    'ip':            'uint32',
    'app':           'uint16',
    'device':        'uint16',
    'os':            'uint16',
    'channel':       'uint16',
    'is_attributed': 'uint8',
    'click_id':      'uint32'
}
train = pd.read_csv(PATH + 'train.csv', nrows=test_nrows, dtype=dtypes,
                    usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']).reset_index()
test = pd.read_csv(PATH + 'test_supplement.csv', nrows=test_nrows, dtype=dtypes,
                    usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']).reset_index()
train['is_train'] = 1
test['is_train'] = 0
test['is_attributed'] = 2
print(train.columns.values)
print(train)

['index' 'ip' 'app' 'device' 'os' 'channel' 'click_time' 'is_attributed'
 'is_train']
   index      ip  app  device  os  channel           click_time  \
0      0   83230    3       1  13      379  2017-11-06 14:32:21   
1      1   17357    3       1  19      379  2017-11-06 14:33:34   
2      2   35810    3       1  13      379  2017-11-06 14:34:12   
3      3   45745   14       1  13      478  2017-11-06 14:34:52   
4      4  161007    3       1  13      379  2017-11-06 14:35:08   

   is_attributed  is_train  
0              0         1  
1              0         1  
2              0         1  
3              0         1  
4              0         1  


# data cleaning

## machine = device + os

In [2]:
import pytz

# set time zone to be Shanghai time and split click_time into day, hour and minute
def data_clean(df):
    tz = pytz.timezone('Asia/Shanghai')
    df['click_time'] = pd.to_datetime(df['click_time']).dt.tz_localize(pytz.utc).dt.tz_convert(tz)
    df['day'] = df['click_time'].dt.day.astype('uint8')
    df['hour'] = df['click_time'].dt.hour.astype('uint8')
    df['minute'] = df['click_time'].dt.minute.astype('uint8')
    df['minute10'] = (df['minute'] / 10).astype('uint8') * 10 # set to 10 minute
    df['hourminute'] = (df['minute'].astype('uint16') + df['hour'].astype('uint16') * 60)
    df['hourminute10'] = (df['minute10'].astype('uint16') + df['hour'].astype('uint16') * 60)
    df['machine'] = 1000 * df['device'] + df['os']
    
data_clean(train)
data_clean(test)
df = train.append(test) # concat train and test

data_type = df.dtypes.to_dict()

label = 'is_attributed'
train_len = train.shape[0]
fdir = './data/'
print(df)

   app  channel  click_id                click_time  day  device  hour  \
0    3      379       NaN 2017-11-06 22:32:21+08:00    6       1    22   
1    3      379       NaN 2017-11-06 22:33:34+08:00    6       1    22   
2    3      379       NaN 2017-11-06 22:34:12+08:00    6       1    22   
3   14      478       NaN 2017-11-06 22:34:52+08:00    6       1    22   
4    3      379       NaN 2017-11-06 22:35:08+08:00    6       1    22   
0    3      379       0.0 2017-11-09 22:23:39+08:00    9       1    22   
1    3      379       1.0 2017-11-09 22:23:51+08:00    9       1    22   
2    3      379       2.0 2017-11-09 22:25:57+08:00    9       1    22   
3    3      379       3.0 2017-11-09 22:26:03+08:00    9       1    22   
4   58      120       4.0 2017-11-09 22:26:41+08:00    9       1    22   

   hourminute  hourminute10  index      ip  is_attributed  is_train  machine  \
0        1352          1350      0   83230              0         1     1013   
1        1353          13

# count

## count the click number for each feature combination

In [5]:
import gc

# Here df is [train test_supp]
def count(df, cols, label, train_len):
    col_name = 'count_' + '_'.join(cols)
    d_cols = list(cols)
    d_cols.append(label)
    count_result = df[d_cols].groupby(by=cols)[[label]].count().rename(index=str, columns={label: col_name}).reset_index()
    type_map = {i: data_type[i] for i in count_result.columns.values if i in data_type.keys()}
    _df = df.merge(count_result.astype(type_map), on=cols, how='left')
    _df[[col_name]][ : train_len].to_csv(fdir + 'train_' + col_name + '.csv', index=False)
    _df[[col_name]][train_len : ].to_csv(fdir + 'test_supplement_' + col_name + '.csv', index=False)
    del _df, count_result
    gc.collect()

patterns = [
    ['app','channel'],
    ['app','device','channel','day','hour'],
    ['app','device','day','hour'],
    ['app','os','channel','day','hour'],
    ['ip','day'],
    ['ip'],
    ['ip','app','device','channel','day'],
    ['ip','app','device','day'],
    ['ip','app','device','os','day','hour'],
    ['ip','app','os','channel'],
    ['ip','app','os','channel','day'],
    ['ip','os'],
    ['app','day','hourminute'],
    ['device','os','day','hourminute10'],
    ['ip','device','os','day','hourminute10']
]

count(df, patterns[0], 'is_attributed', train_len)

   app  channel  click_id                click_time  day  device  hour  \
0    3      379       NaN 2017-11-06 22:32:21+08:00    6       1    22   
1    3      379       NaN 2017-11-06 22:33:34+08:00    6       1    22   
2    3      379       NaN 2017-11-06 22:34:12+08:00    6       1    22   
3   14      478       NaN 2017-11-06 22:34:52+08:00    6       1    22   
4    3      379       NaN 2017-11-06 22:35:08+08:00    6       1    22   
5    3      379       0.0 2017-11-09 22:23:39+08:00    9       1    22   
6    3      379       1.0 2017-11-09 22:23:51+08:00    9       1    22   
7    3      379       2.0 2017-11-09 22:25:57+08:00    9       1    22   
8    3      379       3.0 2017-11-09 22:26:03+08:00    9       1    22   
9   58      120       4.0 2017-11-09 22:26:41+08:00    9       1    22   

   hourminute  hourminute10  index      ip  is_attributed  is_train  machine  \
0        1352          1350      0   83230              0         1     1013   
1        1353          13

# unique count

## group data by certain feature combination and count the number of different values of another feature

In [15]:
import gc

def unique_count(df, cols, train_len):
    col_name = 'nunique_' + '_'.join(cols)
    count_result = df[cols].groupby(by=cols[:-1])[[cols[-1]]].nunique().rename(index=str,\
                                                                               columns={cols[-1]: col_name}).reset_index()
    type_map = {i: data_type[i] for i in count_result.columns.values if i in data_type.keys()}
    _df = df.merge(count_result.astype(type_map), on=cols[:-1], how='left')
    _df[[col_name]][ : train_len].to_csv(fdir + 'train_' + col_name + '.csv', index=False)
    _df[[col_name]][train_len : ].to_csv(fdir + 'test_supplement_' + col_name + '.csv', index=False)
    del _df, count_result
    gc.collect()
    
patterns = [
    ['day','ip','machine'],
    ['day','ip','os'],
    ['day','ip','device'],
    ['day','ip','app'],
    ['day','ip','channel'],
    ['machine','app'],
    ['machine','ip'],
    ['machine','channel'],
]

unique_count(df, ['app','channel','ip'], train_len)

   app  channel  click_id                click_time  day  device  hour  \
0    3      379       NaN 2017-11-06 22:32:21+08:00    6       1    22   
1    3      379       NaN 2017-11-06 22:33:34+08:00    6       1    22   
2    3      379       NaN 2017-11-06 22:34:12+08:00    6       1    22   
3   14      478       NaN 2017-11-06 22:34:52+08:00    6       1    22   
4    3      379       NaN 2017-11-06 22:35:08+08:00    6       1    22   
5    3      379       0.0 2017-11-09 22:23:39+08:00    9       1    22   
6    3      379       1.0 2017-11-09 22:23:51+08:00    9       1    22   
7    3      379       2.0 2017-11-09 22:25:57+08:00    9       1    22   
8    3      379       3.0 2017-11-09 22:26:03+08:00    9       1    22   
9   58      120       4.0 2017-11-09 22:26:41+08:00    9       1    22   

   hourminute  hourminute10  index      ip  is_attributed  is_train  machine  \
0        1352          1350      0   83230              0         1     1013   
1        1353          13

# cumulative count

## give an order number in each feature combination by each feature combination, sorted by [click_time, index, is_train]

In [21]:
import gc

def cum_count(df, cols, train_len):
    col_name = 'cumcount_' + '_'.join(cols)
    result = df[cols].groupby(cols).cumcount().rename(col_name).to_frame()
    result[:train_len].to_csv(fdir + 'train_' + col_name + '.csv', index=False)
    result[train_len:].to_csv(fdir + 'test_supplement_' + col_name + '.csv', index=False)
    del result
    gc.collect()
    
patterns = [
    ['ip','app','device','os','day','hour'],
    ['ip','day'],
    ['app','device','os','day']
]

cum_count(df, ['app', 'channel'], train_len)

   cumcount_app_channel
0                     0
1                     1
2                     2
3                     0
4                     3
0                     4
1                     5
2                     6
3                     7
4                     0

   app  channel  click_id                click_time  day  device  hour  \
0    3      379       NaN 2017-11-06 22:32:21+08:00    6       1    22   
1    3      379       NaN 2017-11-06 22:33:34+08:00    6       1    22   
2    3      379       NaN 2017-11-06 22:34:12+08:00    6       1    22   
3   14      478       NaN 2017-11-06 22:34:52+08:00    6       1    22   
4    3      379       NaN 2017-11-06 22:35:08+08:00    6       1    22   
0    3      379       0.0 2017-11-09 22:23:39+08:00    9       1    22   
1    3      379       1.0 2017-11-09 22:23:51+08:00    9       1    22   
2    3      379       2.0 2017-11-09 22:25:57+08:00    9       1    22   
3    3      379       3.0 2017-11-09 22:26:03+08:00    9       1    2

In [22]:
xxx = pd.read_csv(fdir + 'train_cumcount_app_channel.csv')
print(xxx)

   cumcount_app_channel
0                     0
1                     1
2                     2
3                     0
4                     3


# count ratio

## cols1 count / cols2 count

In [7]:
import gc

def _count(df, cols, label):
    col_name = 'count_ratio_' + '_'.join(cols)
    d_cols = list(cols)
    d_cols.append(label)
    count_result = df[d_cols].groupby(by=cols)[[label]].count().rename(index=str, columns={label: col_name}).reset_index()
    type_map = {i: data_type[i] for i in count_result.columns.values if i in data_type.keys()}
    _df = df.merge(count_result.astype(type_map), on=cols, how='left')
    result = _df[[col_name]].copy()
    del _df, count_result
    gc.collect()
    return result

def count_ratio(df, cols1, cols2, label, train_len):
    col_name = 'count_ratio_' + '_'.join(cols1) + '_' + '_'.join(cols2)
    x1 = _count(df, cols1, label)
    x2 = _count(df, cols2, label)
    x1[col_name] = x1[x1.columns.values[0]] / x2[x2.columns.values[0]] # or = round(x1 / x2, 4)
    result = x1[[col_name]]
    result[:train_len].to_csv(fdir + 'train_' + col_name + '.csv', index=False)
    result[train_len : ].to_csv(fdir + 'test_supplement_' + col_name + '.csv', index=False)
    del x1, x2
    gc.collect()
    
patterns = [
    {'cols1':['ip'], 'cols2':['machine']},
    {'cols1':['ip'], 'cols2':['channel']},
    {'cols1':['machine'], 'cols2':['ip']},
    {'cols1':['app'], 'cols2':['channel']},
    {'cols1':['channel'], 'cols2':['app']}
]

count_ratio(df, ['app'], patterns[0]['cols2'], label, train_len)

   count_ratio_app_machine
0                 1.333333
1                 8.000000
2                 1.333333
3                 0.166667
4                 1.333333
5                 8.000000
6                 1.333333
7                 1.333333
8                 8.000000
9                 1.000000


In [8]:
xxx = pd.read_csv('./data/train_count_ratio_app_machine.csv')
print(xxx.columns.values)

['count_ratio_app_machine']


# cumulative count ratio

## cols cumcount / (cols count-1)

In [49]:
import gc

def _count(df, cols, label):
    col_name = 'count_ratio_' + '_'.join(cols)
    d_cols = list(cols)
    d_cols.append(label)
    count_result = df[d_cols].groupby(by=cols)[[label]].count().rename(index=str, columns={label: col_name}).reset_index()
    type_map = {i: data_type[i] for i in count_result.columns.values if i in data_type.keys()}
    _df = df.merge(count_result.astype(type_map), on=cols, how='left')
    result = _df[[col_name]].copy()
    del _df, count_result
    gc.collect()
    return result

def _cum_count(df, cols):
    col_name = 'cumcount_ratio_' + '_'.join(cols)
    result = df[cols].groupby(cols).cumcount().rename(col_name).to_frame()
    return result.reset_index()[[col_name]]
    
def cum_count_ratio(df, cols, label, train_len):
    col_name = 'cumcount_ratio_' + '_'.join(cols)
    x1 = _cum_count(df, cols)
    x2 = _count(df, cols, label)
    print(x1, x2)
    x1[col_name] = round(x1[x1.columns.values[0]] / (x2[x2.columns.values[0]] - 1), 4).fillna(1.1)
    result = x1[[col_name]]
    result[:train_len].to_csv(fdir + 'train_' + col_name + '.csv', index=False)
    result[train_len : ].to_csv(fdir + 'test_supplement_' + col_name + '.csv', index=False)
    print(result)
    del x1, x2
    gc.collect()
    
patterns = [
    ['ip','day']
]

cum_count_ratio(df, ['app','channel'], label, train_len)

   cumcount_ratio_app_channel
0                           0
1                           1
2                           2
3                           0
4                           3
5                           4
6                           5
7                           6
8                           7
9                           0    count_ratio_app_channel
0                        8
1                        8
2                        8
3                        1
4                        8
5                        8
6                        8
7                        8
8                        8
9                        1
   cumcount_ratio_app_channel
0                      0.0000
1                      0.1429
2                      0.2857
3                      1.1000
4                      0.4286
5                      0.5714
6                      0.7143
7                      0.8571
8                      1.0000
9                      1.1000


In [51]:
xxx = pd.read_csv(fdir + 'train_cumcount_ratio_app_channel.csv')
print(xxx)

   cumcount_ratio_app_channel
0                      0.0000
1                      0.1429
2                      0.2857
3                      1.1000
4                      0.4286
